In [1]:
import pyomo.environ as pyo
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd

In [2]:
world = gpd.read_file("ne_110m_admin_0_countries.zip")

In [3]:
input_data = pd.read_excel("Inputdata.xlsx", index_col="Land")

## Solver

In [4]:
solver = pyo.SolverFactory('gurobi') 

## Model

In [5]:
model = pyo.ConcreteModel()

## Länder als itterable

input_data.index

In [6]:
model.country = input_data.index

## Parameter

In [7]:
# Parameter in Model importieren
model.h2_price =  pyo.Param(model.country, initialize=input_data["H2 Herstellungspreis"].to_dict())
model.h2_pipeline =  pyo.Param(model.country, initialize=input_data["H2 Pipeline Preis"].to_dict())
model.h2_ship =  pyo.Param(model.country, initialize=input_data["H2 Schiff Preis"].to_dict())
model.h2_limit = pyo.Param(model.country, initialize=input_data["H2 Export Limit"].to_dict())

model.nh3_price =  pyo.Param(model.country, initialize=input_data["NH3 Herstellungspreis"].to_dict())
model.nh3_pipeline =  pyo.Param(model.country, initialize=input_data["NH3 Pipeline Preis"].to_dict())
model.nh3_ship =  pyo.Param(model.country, initialize=input_data["NH3 Schiff Preis"].to_dict())
model.nh3_limit = pyo.Param(model.country, initialize=input_data["NH3 Export Limit"].to_dict())

model.ch3oh_price =  pyo.Param(model.country, initialize=input_data["CH3OH Herstellungspreis"].to_dict())
model.ch3oh_pipeline =  pyo.Param(model.country, initialize=input_data["CH3OH Pipeline Preis"].to_dict())
model.ch3oh_ship =  pyo.Param(model.country, initialize=input_data["CH3OH Schiff Preis"].to_dict())
model.ch3oh_limit = pyo.Param(model.country, initialize=input_data["CH3OH Export Limit"].to_dict())

model.h2_price.pprint()

## Variablen

In [8]:
# Variablen definieren
model.h2_amount_ship = pyo.Var(model.country, domain = pyo.NonNegativeReals)
model.h2_amount_pipeline = pyo.Var(model.country, domain = pyo.NonNegativeReals)

model.nh3_amount_ship = pyo.Var(model.country, domain = pyo.NonNegativeReals)
model.nh3_amount_pipeline = pyo.Var(model.country, domain = pyo.NonNegativeReals)

model.ch3oh_amount_ship = pyo.Var(model.country, domain = pyo.NonNegativeReals)
model.ch3oh_amount_pipeline = pyo.Var(model.country, domain = pyo.NonNegativeReals)

sum(model.h2_amount_pipeline[i] for i in model.country) + sum(model.h2_ship[i] for i in model.country)

imported_h2 = sum(model.h2_amount_pipeline[i] for i in model.country) + sum(model.h2_amount_ship[i] for i in model.country)
imported_nh3 = sum(model.nh3_amount_pipeline[i] for i in model.country) + sum(model.nh3_amount_ship[i] for i in model.country)
imported_ch3oh = sum(model.ch3oh_amount_pipeline[i] for i in model.country) + sum(model.ch3oh_amount_ship[i] for i in model.country)

In [9]:
model.h2_amount_to_nh3 = pyo.Var(model.country, domain = pyo.NonNegativeReals)
model.h2_amount_to_ch3oh = pyo.Var(model.country, domain = pyo.NonNegativeReals)

model.pprint()

## Zielfunktion

# Umtauschbarkeit von H2 NH3 CH3OH

In [10]:
h2_to_nh3_eff = 10
h2_to_ch3oh_eff = 20

# Zielfunktion
obj_expr = sum(model.h2_pipeline[i]*model.h2_amount_pipeline[i] + model.h2_ship[i]*model.h2_amount_ship[i] + model.h2_price[i]*(model.h2_amount_pipeline[i] + model.h2_amount_ship[i]) +
               model.nh3_pipeline[i]*model.nh3_amount_pipeline[i] + model.nh3_ship[i]*model.nh3_amount_ship[i] + model.nh3_price[i]*(model.nh3_amount_pipeline[i] + model.nh3_amount_ship[i]) +
               model.ch3oh_pipeline[i]*model.ch3oh_amount_pipeline[i] + model.ch3oh_ship[i]*model.ch3oh_amount_ship[i] + model.ch3oh_price[i]*(model.ch3oh_amount_pipeline[i] + model.ch3oh_amount_ship[i]) for i in model.country)
model.obj = pyo.Objective(sense=pyo.minimize, expr=obj_expr)

# Zielfunktion alt
obj_expr = sum(model.h2_pipeline[i]*model.h2_amount_pipeline[i] + model.h2_ship[i]*model.h2_amount_ship[i] + model.h2_price[i]*(model.h2_amount_pipeline[i] + model.h2_amount_ship[i]) +
               model.nh3_pipeline[i]*model.nh3_amount_pipeline[i] + model.nh3_ship[i]*model.nh3_amount_ship[i] + model.nh3_price[i]*(model.nh3_amount_pipeline[i] + model.nh3_amount_ship[i]) +
               model.ch3oh_pipeline[i]*model.ch3oh_amount_pipeline[i] + model.ch3oh_ship[i]*model.ch3oh_amount_ship[i] + model.ch3oh_price[i]*(model.ch3oh_amount_pipeline[i] + model.ch3oh_amount_ship[i])
               for i in model.country) 
model.obj = pyo.Objective(sense=pyo.minimize, expr=obj_expr)

# siehe GPT

In [11]:
# Zielfunktion neu
obj_expr = sum(
        # Tranportkosten * Menge je Tranportoption + Herstellungspreis * Gesamtmenge
          model.h2_pipeline[i]*model.h2_amount_pipeline[i] + model.h2_ship[i]*model.h2_amount_ship[i] + model.h2_price[i]*(model.h2_amount_pipeline[i] + model.h2_amount_ship[i])
        # Tranportkosten * Menge je Tranportoption + Herstellungspreis * Gesamtmenge
        + model.nh3_pipeline[i]*model.nh3_amount_pipeline[i] + model.nh3_ship[i]*model.nh3_amount_ship[i] + model.nh3_price[i]*(model.nh3_amount_pipeline[i] + model.nh3_amount_ship[i])
        # Umwandlung von Wasserstoff zu Chemikalie
        + model.h2_amount_to_nh3[i] * (h2_to_nh3_eff * (model.h2_amount_pipeline[i] + model.h2_amount_ship[i]) - (model.nh3_amount_pipeline[i] + model.nh3_amount_ship[i]))
        # Tranportkosten * Menge je Tranportoption + Herstellungspreis * Gesamtmenge                             
        + model.ch3oh_pipeline[i]*model.ch3oh_amount_pipeline[i] + model.ch3oh_ship[i]*model.ch3oh_amount_ship[i] + model.ch3oh_price[i]*(model.ch3oh_amount_pipeline[i] + model.ch3oh_amount_ship[i])
        for i in model.country) 
model.obj = pyo.Objective(sense=pyo.minimize, expr=obj_expr)

# müssen noch gemacht werden

## Nebenbedingungen

In [12]:
# Bedarfsdeckung
model.demand_constraints = pyo.ConstraintList()

con_demand_h2_lhs = sum(model.h2_amount_pipeline[i] + model.h2_amount_ship[i] for i in model.country) - sum(model.h2_amount_to_nh3[i] for i in model.country)
con_demand_h2_rhs = 1000000
model.demand_constraints.add(con_demand_h2_lhs >= con_demand_h2_rhs)

con_demand_nh3_lhs = sum(model.nh3_amount_pipeline[i] + model.nh3_amount_ship[i] for i in model.country) + h2_to_nh3_eff*sum(model.h2_amount_to_nh3[i] for i in model.country)
con_demand_nh3_rhs = 1000000
model.demand_constraints.add(con_demand_nh3_lhs >= con_demand_nh3_rhs)

con_demand_ch3oh_lhs = sum(model.ch3oh_amount_pipeline[i] + model.ch3oh_amount_ship[i] for i in model.country)
con_demand_ch3oh_rhs = 1000000
model.demand_constraints.add(con_demand_ch3oh_lhs >= con_demand_ch3oh_rhs)

# Exportlimits
model.export_constraints = pyo.ConstraintList()

for i in model.country:
    con_export_h2_lhs = model.h2_amount_pipeline[i] + model.h2_amount_ship[i]
    con_export_h2_rhs = model.h2_limit[i]
    model.export_constraints.add(con_export_h2_lhs <= con_export_h2_rhs)
    model.export_constraints.add(0 <= con_export_h2_lhs)
    
    con_export_nh3_lhs = model.nh3_amount_pipeline[i] + model.nh3_amount_ship[i]
    con_export_nh3_rhs = model.nh3_limit[i]
    model.export_constraints.add(con_export_nh3_lhs <= con_export_nh3_rhs)
    model.export_constraints.add(0 <= con_export_nh3_lhs)
    
    con_export_ch3oh_lhs = model.ch3oh_amount_pipeline[i] + model.ch3oh_amount_ship[i]
    con_export_ch3oh_rhs = model.ch3oh_limit[i]
    model.export_constraints.add(con_export_ch3oh_lhs <= con_export_h2_rhs)
    model.export_constraints.add(0 <= con_export_ch3oh_lhs)


# Umwandlung
model.exchange_constraints = pyo.ConstraintList()

for i in model.country:
    con_exchange = (model.h2_amount_pipeline[i] + model.h2_amount_ship[i]) >= model.h2_amount_to_nh3[i] + model.h2_amount_to_ch3oh[i]
    model.exchange_constraints.add(con_exchange)


model.pprint()

In [ ]:
# Create a model instance and optimize
instance = model.create_instance()

solver.options['NonConvex'] = 2
results = solver.solve(instance)
#instance.display()

In [ ]:
instance.display()